In [ ]:
def signals_sec312(Noversampl, oof0, ofcont0, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise, SNR, ndepth, AVG):
#def signals_sec312(_):
    
    f = np.linspace(0, 1, ndepth)
    depth0 = f*LB/2
    
    sstep = nptslambda/Noversampl
    ph0 = ph0/2/np.pi
    ph = int(nptslambda * ph0)
    
    snr1 = np.zeros([ndepth, AVG])
    
    for avg in range(0, AVG):
        
        oof = oof0
        ofcont = ofcont0
        sf = np.zeros([len(oof), ndepth], dtype=complex)
        sf_snr = np.zeros([len(oof), ndepth], dtype=complex)
        
        for n in range(0, ndepth):
            depth = depth0[n]

            #FRINGE
            er = np.sqrt(amp) #reference field
            es = R*np.sqrt(amp)*np.exp(1j*(2*np.pi*oof*(2*depth/c))) #sample field

            #I/Q detection and background subtraction
            if BG:
                sfI = np.square(np.abs(er + es)) - (np.square(er) + np.square(R*np.sqrt(amp)))
                sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))  - (np.square(er) + np.square(R*np.sqrt(amp)))
            else:
                sfI = np.square(np.abs(er + es))
                sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))

            #add noise
            sfI = sfI + np.random.randint(-10000,10000,len(oof))/10000*noise
            sfQ = sfQ + np.random.randint(-10000,10000,len(oof))/10000*noise

            if IQ:
                sf[:, n] = sfI + 1j*sfQ
            else:
                sf[:, n] = sfI    

            #SNR
            if SNR:
                sfI_snr = np.square(np.abs(er)) + np.random.randint(-10000,10000,len(oof))/10000*noise - np.square(er) 
                sfQ_snr = np.square(np.abs(er)) + np.random.randint(-10000,10000,len(oof))/10000*noise - np.square(er)
                sf_snr[:, n] = sfI_snr + 1j*sfQ_snr   

        #ANALOG FILTER
        if FILT:
            if FiltType == 1:
                b, a = signal.butter(Order, FILTFACT/nptslambda, 'low') 
            else:
                b, a = signal.cheby1(Order, 0.5, FILTFACT/nptslambda, 'low')

            sfilt = signal.filtfilt(b, a, sf, axis=0);

            if SNR:
                sfilt_snr = signal.filtfilt(b, a, sf_snr, axis=0)
        else:
            sfilt = sf

            if SNR:
                sfilt_snr = sf_snr

        #SAMPLING
        #cropping the middle one of three sweep
        ind = int(len(amp)/3)
        amp0 = amp[ind+1:ind+1+npts]; 

        oof = oof[ind+1:ind+1+npts];
        ofcont = ofcont[ind+1:ind+1+npts];

        sf = sf[ind+1:ind+1+npts, :];

        #get acquisition phase
        if AutoPhase: 
            maxsearch = np.zeros(50)
            count_test = 0
            phase_test = np.linspace(0,0.98,50)
            for n in range(0,50):
                count_test = count_test + 1

                ph_test = int(nptslambda*phase_test[n])

                s_test = np.roll(sfilt, ph_test, axis=0)
                inds = int(ind+1)
                inde = int(ind+1+npts)
                indstep = int(sstep)

                s_test = s_test[inds:inde:indstep, ndepth-1]

                S_test = np.abs(np.fft.fftshift(np.fft.ifft(s_test,nfft, axis=0), axes=0)) 

                mag = max(S_test)
                maxsearch[n] = mag

            maxind = np.argmax(maxsearch)
            ph = int(nptslambda*phase_test[maxind])

        sfilt_roll = np.roll(sfilt, ph, axis=0)  #apply clock phase 

        inds = int(ind+1)
        inde = int(ind+1+npts)
        indstep = int(sstep)
        ss = sfilt_roll[inds:inde:indstep, :] # DAQ sampling
        ss = np.roll(ss, 1, axis=0) # To make fringe signal before and after sampling align in plot. Reduces confusion. Doesn't change anything. 

        sfilt_plot = sfilt[ind+1:ind+1+npts, :] # just for plotting fringe before sampling

        if SNR:
            ss_snr = sfilt_snr[inds:inde:indstep, :] 

        #DEPTH VECTOR
        inds = 0
        inde = int(len(ofcont))
        indstep = int(sstep)
        ofcont_sampled = ofcont[inds:inde:indstep]            #%sampled frequency vector
        df_sampled = ofcont_sampled[1]-ofcont_sampled[0]      #frequency resolution of sampled field
        dt_sampled = 1/(df_sampled*nfft);
        temp_vec = np.linspace(-nfft/2,nfft/2,nfft)
        dsampled = temp_vec*dt_sampled*c/2                    #depth vector sampled sampled fringe

        #FFT
        Ss0 = np.fft.fftshift(np.fft.ifft(ss, nfft, axis=0), axes=0)
        Ssout = Ss0
        Ss = np.square(np.abs(Ssout))

        if SNR:
            snr0 = np.zeros(ndepth)

            Ss_snr = np.fft.fftshift(np.fft.ifft(ss_snr, nfft, axis=0), axes=0)
            Ss_snr = np.square(np.abs(Ss_snr))

            #mag = np.zeros(ndepth)
            #nois = np.zeros(ndepth)

            for n in range(0, ndepth):
                mag = np.max(Ss[:, n])
                magind = np.argmax(Ss[:, n])
                
                if magind < nfft-500 and magind > nfft/4:
                    nois = np.mean(Ss_snr[magind-450:magind+450, :], axis=0) # average across 800 points of the noise Aline at PSF location
                else:
                    if magind < nfft/4:  # last PSF is at edge of baseband. Its max. may be detected on other side. Range for averaging needs to be adjusted in that case.
                        magind = nfft-magind
                    nois1 = Ss_snr[magind-450:magind-1, :]
                    nois2 = Ss_snr[nfft-magind+1:nfft-magind+450, :]
                    nois3 = np.concatenate((nois1, nois2), axis=0)
                    ii = np.isfinite(nois3)
                    nois3 = nois3[ii]
                    nois = np.mean(nois3, axis=0)
                    
                nois = np.mean(nois) # average across all generated noise Alines
                snr0[n] = mag/nois
        else:
            snr0 = 0            
        
        if SNR:
            snr1[:, avg] = snr0
    
    snr = np.mean(snr1, axis=1)
    return(snr)